# Coulomb gauge transformation

### Simulation parameters

In [ ]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 64 
# N = 512  
tau_sim = 1.0     
DTS = 4     

# Glasma fields
su_group = 'su3'
Qs = 2     

ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

nevents = 1

### Environment variables

In [ ]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,4,5,6,7"

os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.coulomb as coulomb

Using CUDA
Using SU(3)
Using double precision


### Simulate glasma

In [ ]:
niter = 10

# alpha abelian
factor_alpha = 1.0
alpha = 4.0 * (np.sin((np.pi * (N-1)) / N) ** 2 + np.sin((np.pi * (N-1)) / N) ** 2) * factor_alpha

In [4]:
from numba.core.errors import NumbaPerformanceWarning
import warnings

warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

In [ ]:
import pickle
from tqdm import tqdm

# Simulation routine
# def simulate(): 

# Derived parameters
a = L / N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)

# Initialize Glasma fields
s = core.Simulation(N, DT, g)
va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
initial.init(s, va, vb)

# with tqdm(total=maxt) as pbar:
#     for t in range(maxt):      
#         #TODO: evolve the glasma fields, at the moment consider only 1 evolution step

#         # Evolve Glasma fields
#         core.evolve_leapfrog(s)

#         pbar.update(1)

core.evolve_leapfrog(s)

coul = coulomb.CoulombGaugeTransf(s)
coulomb.init_gauge_transform(coul)

for iter in range(niter):
    print('niter=', iter+1)

    coulomb.gauge_transform(coul, alpha)

    theta = coul.theta.copy()
    print('theta=', theta)

if use_cuda:
    cuda.current_context().deallocations.clear()

niter= 1
theta= 1.201135474612858
niter= 2
theta= 1.5487818332170829e-06
niter= 3
theta= 4.098232641920193e-20
niter= 4
theta= 1.416067715847816e-40
niter= 5
theta= 5.987293096491756e-68
niter= 6
theta= 2.947511351147737e-102
niter= 7
theta= 2.06459900877772e-143
niter= 8
theta= 2.0394026917280923e-191
niter= 9
theta= 2.5786226100769e-246
niter= 10
theta= 4.1896887609857804e-308
